In [14]:
import polars as pl
import polars.selectors as cs
import plotly.express as px
import os

data_dir = './Provided_Data/'
df_list = []
csv_list =[f for f in os.listdir(data_dir) if f.endswith('csv')]
for file in csv_list:
    print(file)
    df = (
        pl.read_csv(
            f'{data_dir}/{file}',
            schema_overrides={
                'Grade': pl.String,
            },
            ignore_errors=True,
        )
        .rename(    
            lambda c: 
                c.upper()          # all column names to upper case
                .replace(' ', '_') # replace blanks with underscores
                .replace(r'(', '') # replace left parens with blank text
                .replace(r')', '') # replace left parens with blank text
                .replace(r'+', '_') # replace plus signs with underscores
        )
    )
    print(df.columns)
    print(df.sample(2).glimpse())

df_all = (
    pl.concat(df, how='diagonal')
)
# df = (
#     pl.read_csv(
#         'LockerNYC_Reservations_20250903.csv',
#     )
#     .rename(    
#         lambda c: 
#             c.upper()          # all column names to upper case
#             .replace(' ', '_') # replace blanks with underscores
#             .replace(r'(', '') # replace left parens with underscores
#             .replace(r')', '') # replace left parens with underscores
#     )
#     .select(
#         ['TYPE', 'DELIVERED',
#         'RECEIVED',
#         'LOCKER_NAME', 'LOCKER_BOX_DOOR', 'LOCKER_SIZE', 
#         'PICKUP_DURATION', 'DELIVERY_DURATION', 
#         'LOCATION_TYPE', 'ADDRESS', 'LATITUDE', 'LONGITUDE', 
#         'BOROUGH',
#         'RECEIVE_DATE', 'CREATED_DATE', 'DELIVERY_DATE', 'WITHDRAW_DATE', 'EXPIRE_DATE',
#         ],
#     )
#     .with_columns(cs.ends_with('_DATE').str.to_datetime(format="%m/%d/%Y %H:%M"))
#     .with_columns(ZIP_CODE = pl.col('ADDRESS').str.split(' ').list.last())
#     .with_columns(ADDRESS = pl.col('ADDRESS').str.split(',').list.first())
# )
# print(df.sample(3))


2006-2011_NYS_Math_Test_Results_By_Grade_-_Boro_-_By_Race-_Ethnicity.csv
['BOROUGH', 'GRADE', 'YEAR', 'CATEGORY', 'NUMBER_TESTED', 'MEAN_SCALE_SCORE', 'LEVEL_1_#', 'LEVEL_1_%', 'LEVEL_2_#', 'LEVEL_2_%', 'LEVEL_3_#', 'LEVEL_3_%', 'LEVEL_4_#', 'LEVEL_4_%', 'LEVEL_3_4_#', 'LEVEL_3_4_%']
Rows: 2
Columns: 16
$ BOROUGH          <str> 'STATEN ISLAND', 'QUEENS'
$ GRADE            <str> '3', '7'
$ YEAR             <i64> 2006, 2006
$ CATEGORY         <str> 'White', 'Asian'
$ NUMBER_TESTED    <i64> 2426, 5139
$ MEAN_SCALE_SCORE <i64> 693, 672
$ LEVEL_1_#        <i64> 91, 259
$ LEVEL_1_%        <f64> 3.8, 5.0
$ LEVEL_2_#        <i64> 161, 952
$ LEVEL_2_%        <f64> 6.6, 18.5
$ LEVEL_3_#        <i64> 1167, 2508
$ LEVEL_3_%        <f64> 48.1, 48.8
$ LEVEL_4_#        <i64> 1007, 1420
$ LEVEL_4_%        <f64> 41.5, 27.6
$ LEVEL_3_4_#      <i64> 2174, 3928
$ LEVEL_3_4_%      <f64> 89.6, 76.4

None
2006-2011_NYS_Math_Test_Results_by_Grade_-_Citywide_-_by_Gender.csv
['GRADE', 'YEAR', 'CATEGORY', 'NUMBE

ValueError: Series only supports 'vertical' concat strategy

In [ ]:
(
    df
    .filter(pl.col('CREATED_DATE') != pl.col('RECEIVE_DATE'))
    .filter(pl.col('PICKUP_DURATION').is_not_null())
    .sample(10)
    .glimpse()
)

In [ ]:
(
    df
    .filter(pl.col('CREATED_DATE') != pl.col('RECEIVE_DATE'))
    .filter(pl.col('PICKUP_DURATION').is_not_null())
    .select('CREATED_DATE', 'DELIVERY_DATE', 'RECEIVE_DATE','WITHDRAW_DATE', 'EXPIRE_DATE' )
    .sample(5)
    .transpose(include_header=True)
    .sort('column_0')
)



# .str.strptime(pl.Datetime, fmt="%m/%d/%Y %H:%M")


In [ ]:
for c in df.columns:
    print(
        c, 
        df[c]
        .value_counts()
        .with_columns(
            PCT = (100*pl.col('count')/pl.col('count').sum()).cast(pl.Float32)
        )
    )

In [ ]:
print(list(df.columns))
print()

In [ ]:
print(sorted(list(set(df['BOROUGH']))))

In [ ]:
fig = px.scatter_map(
    df,
    lat = 'LATITUDE',
    lon = 'LONGITUDE',
    color='BOROUGH'
)
fig.show()

In [ ]:
group_by_cols = ['LOCKER_NAME', 'ADDRESS', 'LOCATION_TYPE', 'LATITUDE', 'LONGITUDE', 'BOROUGH']
df_group_by = (
    df
    .group_by(group_by_cols).len()
    .rename({'len':  'COUNT'})
    .sort('COUNT')
)
df_group_by
# df_group_by['len'].sum()

In [ ]:
df.columns